In [3]:
!pip install -q kaggle pandas numpy

In [7]:
import os, json, getpass, pathlib
pathlib.Path("/root/.kaggle").mkdir(exist_ok=True)
if not os.path.exists("/root/.kaggle/kaggle.json"):
    KAGGLE_USERNAME = input("KAGGLE_USERNAME: ")
    KAGGLE_KEY = getpass.getpass("KAGGLE_KEY: ")
    with open("/root/.kaggle/kaggle.json", "w") as f:
        json.dump({"username": KAGGLE_USERNAME, "key": KAGGLE_KEY}, f)
!chmod 600 /root/.kaggle/kaggle.json

In [8]:
!mkdir -p /content/raw
%cd /content/raw
!kaggle competitions download -c demand-forecasting-kernels-only -p .
!unzip -o demand-forecasting-kernels-only.zip
!ls -al

/content/raw
  0% 0.00/3.29M [00:00<?, ?B/s]
100% 3.29M/3.29M [00:00<00:00, 877MB/s]
Archive:  demand-forecasting-kernels-only.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               
total 21656
drwxr-xr-x 2 root root     4096 Nov 10 05:25 .
drwxr-xr-x 1 root root     4096 Nov 10 05:25 ..
-rw-r--r-- 1 root root  3454726 Dec 15  2019 demand-forecasting-kernels-only.zip
-rw-r--r-- 1 root root   393899 Dec 15  2019 sample_submission.csv
-rw-r--r-- 1 root root   975309 Dec 15  2019 test.csv
-rw-r--r-- 1 root root 17333449 Dec 15  2019 train.csv


In [ ]:
import pandas as pd

df = pd.read_csv("/content/raw/train.csv")     
df["date"] = pd.to_datetime(df["date"])

# 월요일 시작 주차로 합계 
weekly = (
    df.groupby(["store","item", pd.Grouper(key="date", freq="W-MON")])["sales"]
      .sum()
      .reset_index()
      .rename(columns={
          "store":"store_id",
          "item":"product_id",
          "date":"target_date",
          "sales":"actual_order_qty"
      })
)

# 창고/상품 도메인 간단 라벨 
product_map = {1:"TV",2:"냉장고",3:"세탁기",4:"에어컨",5:"전자레인지",
               6:"청소기",7:"공기청정기",8:"커피머신",9:"오븐",10:"식기세척기"}
weekly["product_name"] = weekly["product_id"].map(product_map).fillna("기타가전")
weekly["warehouse_id"] = (weekly["store_id"] % 3) + 1  # 창고 3개 가정

# 전체와 샘플 저장 
weekly.to_csv("/content/weekly_sales.csv", index=False)
weekly.sample(300, random_state=42).to_csv("/content/weekly_sales_sample.csv", index=False)

len(weekly), weekly.head()


(130500,
    store_id  product_id target_date  actual_order_qty product_name  \
 0         1           1  2013-01-07                83           TV   
 1         1           1  2013-01-14                68           TV   
 2         1           1  2013-01-21                76           TV   
 3         1           1  2013-01-28                73           TV   
 4         1           1  2013-02-04                89           TV   
 
    warehouse_id  
 0             2  
 1             2  
 2             2  
 3             2  
 4             2  )